Stv88

Stv90

Stv91

Stv94

Stv104

Stv_15	GCCCGTATTCCCGCTTGCGAGTAGGGCAAT
Stv_24	GGGAAACGGTTGTCGCTACCGGCTTTGCTA
Stv_47	ATCGGAACGGTGACTCGGCTCAGTCGCGGC
Stv_52	GCACCGGTCGGGATTCCGTAGGCACGCCGA
Stv_77	gcgtcgatcggagaagcgctcgggtatgac
Stv_124	atggcacccactcgaggttactccgaaagg

In [2]:
%run "E:\Users\puzheng\Documents\Startup_py3.py"
sys.path.append(r"E:\Users\puzheng\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import library_tools
from ImageAnalysis3.library_tools import _readout_folder
from ImageAnalysis3.library_tools import readouts


In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord

In [6]:
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts'


records = [SeqRecord(Seq("GCCCGTATTCCCGCTTGCGAGTAGGGCAAT".upper()), id='Stv_15', description=''),
           SeqRecord(Seq("GGGAAACGGTTGTCGCTACCGGCTTTGCTA".upper()), id='Stv_24', description=''),
           SeqRecord(Seq("ATCGGAACGGTGACTCGGCTCAGTCGCGGC".upper()), id='Stv_47', description=''),
           SeqRecord(Seq("GCACCGGTCGGGATTCCGTAGGCACGCCGA".upper()), id='Stv_52', description=''),
           SeqRecord(Seq("gcgtcgatcggagaagcgctcgggtatgac".upper()), id='Stv_77', description=''),
           SeqRecord(Seq("atggcacccactcgaggttactccgaaagg".upper()), id='Stv_124', description=''),
]

# save temp
save_filename = os.path.join(readout_folder, '20200121_extend_stv.fasta')
with open(save_filename, 'w') as _output_handle:
    print(f"-- saving filtered readouts to file: {save_filename}")
    SeqIO.write(records, _output_handle, "fasta")

-- saving filtered readouts to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts\20200121_extend_stv.fasta


In [7]:
binding_sites = []
for _record in records:
    _site = _record[-20:].reverse_complement()
    _site.id = _record.id+"_last20_site"
    _site.description=''
    binding_sites.append(_site)
save_filename = os.path.join(readout_folder, '20200121_extend_stv_binding_sites.fasta')
with open(save_filename, 'w') as _output_handle:
    print(f"-- saving filtered readouts to file: {save_filename}")
    SeqIO.write(binding_sites, _output_handle, "fasta")

-- saving filtered readouts to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts\20200121_extend_stv_binding_sites.fasta


In [8]:
records[0].reverse_complement

<bound method SeqRecord.reverse_complement of SeqRecord(seq=Seq('GCCCGTATTCCCGCTTGCGAGTAGGGCAAT'), id='Stv_15', name='<unknown name>', description='', dbxrefs=[])>

In [9]:
# load readouts
readout_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Readouts'
readout_start = 0

readouts = library_tools.probes.load_readouts(1356, 
                                              _type='designed_readouts', 
                                              _readout_folder=readout_folder, )

In [10]:
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
ndb_adaptor_file = os.path.join(adaptor_folder, 'NDB_adaptors.fasta')

saved_readouts = library_tools.readouts.Screen_seqs_against_fasta(records, ndb_adaptor_file)

- Screen sequences against given fasta file:\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\NDB_adaptors.fasta
-- 6 sequences loaded.
-- constructing reference table for fasta file
Mapping no. of seqs: 1283
-- 6 sequences kept by allowing hits:0


In [11]:
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
stv_adaptor_file = os.path.join(adaptor_folder, 'Stv_adaptors.fasta')

saved_readouts = library_tools.readouts.Screen_seqs_against_fasta(records, stv_adaptor_file)

- Screen sequences against given fasta file:\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors\Stv_adaptors.fasta
-- 6 sequences loaded.
-- constructing reference table for fasta file
Mapping no. of seqs: 75
-- 6 sequences kept by allowing hits:0


# generate adaptors and save

In [12]:
# load readout sites
adaptor_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Adaptors'
readout_site_file = os.path.join(adaptor_folder, 'Readout_sites.fasta')
readout_sites = []
with open(readout_site_file, 'r') as _rd_handle:
    for _readout in SeqIO.parse(_rd_handle, "fasta"):
        readout_sites.append(_readout)
print(readout_sites)

[SeqRecord(seq=Seq('TTTGCACTGCCGTCCTTGAC', SingleLetterAlphabet()), id='Stv_82', name='Stv_82', description='Stv_82 cy7 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('GATCCGATTGGAACCGTCCC', SingleLetterAlphabet()), id='Stv_1', name='Stv_1', description='Stv_1 cy5 rev-com_last20', dbxrefs=[]), SeqRecord(seq=Seq('TGCGAACTGTCCGGCTTTCA', SingleLetterAlphabet()), id='Stv_79', name='Stv_79', description='Stv_79 cy3 rev-com_last20', dbxrefs=[])]


In [16]:
reload(ia.library_tools.readouts)
from ImageAnalysis3.library_tools.readouts import Generate_adaptors

saved_adaptors = Generate_adaptors(saved_readouts, readout_sites)

In [22]:
# generate csv file to order in IDT
import csv

with open(os.path.join(adaptor_folder, '20200121_extend_stv_adaptors.csv'), 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', lineterminator='\n',
                       quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header
    _header = ['Name', 'Sequence', 'Scale', 'Purification']
    csvwriter.writerow(_header)
    # write sequence
    for _adaptor in saved_adaptors:
        _info = [_adaptor.id, str(_adaptor.seq), '25nm', 'STD']
        csvwriter.writerow(_info)

In [20]:
with open(os.path.join(adaptor_folder, '20200121_extend_stv_adaptors.fasta'), 'w') as _output_handle:
    SeqIO.write(saved_adaptors, _output_handle, "fasta")